# Receipt OCR  

### ressources

http://asprise.com/ocr/api/docs/html/receipt-ocr.html <br/>
https://asprise.com/receipt-ocr-data-capture-api/extract-text-reader-scanner-index.html

In [26]:
import os
import requests
import pandas as pd
import json
from googletrans import Translator


print("=== Python Receipt OCR using API")

=== Python Receipt OCR using API


### OCRing the receipt

#### Read jpg file and save into json

In [40]:
OverAsked = False
receiptOcrEndpoint = 'https://ocr.asprise.com/api/v1/receipt' # Receipt OCR API endpoint
folder_path_img = "receipts/jpg" #folder path
folder_path_json = "receipts/json" 

#iterate over the files in the folder 
if OverAsked == False:
  for filename in os.listdir(folder_path_img):
    if filename.endswith(".jpg"):
        image_path = os.path.join(folder_path_img, filename)
        r = requests.post(receiptOcrEndpoint, data = { \
        'client_id': 'TEST',        # Use 'TEST' for testing purpose \
        'recognizer': 'auto',       # can be 'US', 'CA', 'JP', 'SG' or 'auto' \
        'ref_no': 'ocr_python_123', # optional caller provided ref code \
        }, \
        files = {"file": open(image_path, "rb")})

        # Save the json response to a file
        json_filename = os.path.splitext(filename)[0] + ".json"
        json_path = os.path.join(folder_path_json, json_filename)        
        with open(json_path, "w") as outfile:
            json.dump(r.json(), outfile)
            #https://www.geeksforgeeks.org/reading-and-writing-json-to-a-file-in-python/

        print(f"Processed {filename} - Result saved to {json_filename}")
        print(r.text)

Processed 1.jpg - Result saved to 1.json
{
  "ref_no" : "ocr_python_123",
  "success" : false,
  "message" : "Daily quota exceeded. Try again in a few hours or contact us to increase the quota: ocr@asprise.com",
  "get_help" : "Please contact support@asprise.com for assistance."
}
Processed 2.jpg - Result saved to 2.json
{
  "ref_no" : "ocr_python_123",
  "success" : false,
  "message" : "Daily quota exceeded. Try again in a few hours or contact us to increase the quota: ocr@asprise.com",
  "get_help" : "Please contact support@asprise.com for assistance."
}
Processed 3.jpg - Result saved to 3.json
{
  "ref_no" : "ocr_python_123",
  "success" : false,
  "message" : "Daily quota exceeded. Try again in a few hours or contact us to increase the quota: ocr@asprise.com",
  "get_help" : "Please contact support@asprise.com for assistance."
}
Processed 4.jpg - Result saved to 4.json
{
  "ref_no" : "ocr_python_123",
  "success" : false,
  "message" : "Daily quota exceeded. Try again in a few hou

In [42]:
with open("test.json", "r") as openfile:
    #reading from json file
    json_object = json.load(openfile)

print(json_object)

{'request_id': 'P_218.186.139.27_kl96a7ie_o9k', 'ref_no': 'AspDemo_1613550458205_154', 'file_name': 'SG-01s.jpg', 'request_received_on': 1613550460551, 'success': True, 'image_width': 1200, 'image_height': 1600, 'image_rotation': -0.006, 'recognition_completed_on': 1613550461387, 'receipts': [{'merchant_name': "McDonald's", 'merchant_address': '600 @ Toa Payoh #01-02, Singapore 319515', 'merchant_phone': '62596362', 'merchant_website': None, 'merchant_tax_reg_no': 'M2-0023981-4', 'merchant_company_reg_no': None, 'merchant_logo': '/ocr/api/img/demo/logos/mcd.jpg', 'region': None, 'mall': '600 @ Toa Payoh', 'country': 'SG', 'receipt_no': '002201330026', 'date': '2016-01-13', 'time': '15:49', 'items': [{'amount': 2.95, 'description': 'Med Ice Lemon Tea', 'flags': '', 'qty': 1, 'remarks': None, 'unitPrice': None}, {'amount': 2.4, 'description': 'Coffee with Milk', 'flags': '', 'qty': 1, 'remarks': None, 'unitPrice': None}], 'currency': 'SGD', 'total': 5.35, 'subtotal': None, 'tax': 0.35, '

### Find all the items in the receipt

In [5]:
if json_object["receipts"] is not None: 
    receipt_object = json_object["receipts"]
    # Access the "items" list within the "receipts" object
    items = json_object["receipts"][0]["items"]
    total = json_object["receipts"][0]["total"]
    taxes = json_object["receipts"][0]["tax"]
    print("items: ",items)
    print("total: ",total)
    print("taxes: ",taxes)

else:
    print("Error is"+json_object["message"])

items:  [{'amount': 2.95, 'description': 'Med Ice Lemon Tea', 'flags': '', 'qty': 1, 'remarks': None, 'unitPrice': None}, {'amount': 2.4, 'description': 'Coffee with Milk', 'flags': '', 'qty': 1, 'remarks': None, 'unitPrice': None}]
total:  5.35
taxes:  0.35


### Translation

Ressources : https://www.thepythoncode.com/article/translate-text-in-python

In [6]:
#init the Google API translator
translator = Translator()

In [7]:
for item in items: 
    description = item["description"]
    translation = translator.translate(description, dest="fr")
    item["description"] = translation.text
    print("Description : ", item["description"])

Description :  Thé au citron à glace Med
Description :  Café au lait


### Product

In [8]:
#iterate over each item and add them if flaf != none
products = []

for item in items:
    flag = item["flags"]
    if flag != "":
        products.append(item)
    #elif :
        #find total price
    else:
        #nothing to do
        print()
print(products)



[]


### Pandas dataframe

In [15]:
items_df = pd.DataFrame(items)
products_df = pd.DataFrame(products)

In [25]:
export_csv = pd.DataFrame(products_df[["description", "amount"]])
print(export_csv)
export_csv.to_csv('receipts/export_data.csv', index=False, sep=";")

                 description  amount
0  Thé au citron à glace Med    2.95
1               Café au lait    2.40


## All in one

In [49]:
receipt_object_all = []
items_all = []
total_all = []
taxes_all = []
products_all = []

#init the Google API translator
translator_all = Translator()

for filename in os.listdir(folder_path_json):
    if filename.endswith(".json"):
        json_path = os.path.join(folder_path_json, filename)
        #Read all json in json folder
        with open(json_path, "r") as openfile:
            #reading from json file
            json_object_all = json.load(openfile)
        print(json_object_all)

        if json_object_all["success"] != False:
            #Sort the items, ... in the reicept
            if json_object_all["receipts"] is not None: 
                receipt_object_all.append(json_object["receipts"])
                # Access the "items" list within the "receipts" object
                items_all.append(json_object["receipts"][0]["items"])
                print("items: ",items)
                total_all.append(json_object["receipts"][0]["total"])
                print("total: ",total_all)
                taxes_all.append(json_object["receipts"][0]["tax"])
                print("taxes: ",taxes_all)

            else:
                print("Error is"+json_object["message"])

            #translate all the descriptions
            for item in items_all: 
                description_all = item["description"]
                translation_all = translator_all.translate(description, dest="fr")
                item["description"] = translation_all.text
                print("Description : ", item["description"])

            #find all the product, only the product -> find with tag ?
            for item in items_all:
                flag = item["flags"]
                if flag != "":
                    products_all.append(items_all)
            print(products_all)

if items_all and products_all is not None:
    #Convert into df and csv                
    items_df_all = pd.DataFrame(items_all)
    products_df_all = pd.DataFrame(products_all)

    export_csv_all = pd.DataFrame(products_df_all[["description", "amount"]])
    print(export_csv_all)
    export_csv_all.to_csv('receipts/export_data_all.csv', index=False, sep=";")


{'ref_no': 'ocr_python_123', 'success': False, 'message': 'Daily quota exceeded. Try again in a few hours or contact us to increase the quota: ocr@asprise.com', 'get_help': 'Please contact support@asprise.com for assistance.'}
{'ref_no': 'ocr_python_123', 'success': False, 'message': 'Daily quota exceeded. Try again in a few hours or contact us to increase the quota: ocr@asprise.com', 'get_help': 'Please contact support@asprise.com for assistance.'}
{'ref_no': 'ocr_python_123', 'success': False, 'message': 'Daily quota exceeded. Try again in a few hours or contact us to increase the quota: ocr@asprise.com', 'get_help': 'Please contact support@asprise.com for assistance.'}
{'ref_no': 'ocr_python_123', 'success': False, 'message': 'Daily quota exceeded. Try again in a few hours or contact us to increase the quota: ocr@asprise.com', 'get_help': 'Please contact support@asprise.com for assistance.'}
